In [1]:
import gensim
import pandas as pd
import os
import re
import nltk
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings('ignore')

C:\Users\Carlo\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Labelled Words

In [2]:
rootdir = "data"
positive_words = []
negative_words = []
neutral_words = []

for directories, subdirs, files in os.walk(rootdir):
    if (os.path.split(directories)[1] == '1' or os.path.split(directories)[1] == '2' or os.path.split(directories)[1] == '3' or os.path.split(directories)[1] == '4'):
        for filename in files:
            if (filename == 'positive.txt'):
                with open(os.path.join(directories, filename)) as f:
                    for line in f:
                        positive_words.append(line.strip())
            if (filename == 'negative.txt'):
                with open(os.path.join(directories, filename)) as f:
                    for line in f:
                        negative_words.append(line.strip())
            if (filename == 'neutral.txt'):
                with open(os.path.join(directories, filename), encoding='latin-1') as f:
                    for line in f:
                        neutral_words.append(line.strip())

## Word2Vec

In [3]:
df = pd.read_csv('data/posts_data.csv')
data = df['Body']
data.dropna(inplace=True)
data.drop_duplicates(inplace=True)

In [4]:
# Stop words 
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))
fil_stop = []
with open('assets/stop_words_ph.txt') as f:
    for line in f:
        fil_stop.append(line.rstrip('\n')) 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Carlo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
nltk.download('wordnet')

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Carlo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
text_data = []
counter = 0
index = 0

for line in data:
    tokens = gensim.utils.simple_preprocess(line)
    tokens = [re.sub('\#ADMUFreedomWall\ \d+', '', sent) for sent in tokens]
    tokens = [re.sub('\#ADMUFreedomWall\d+', '', sent) for sent in tokens]
    tokens = [word for word in tokens if word not in fil_stop]
    tokens = [word for word in tokens if word not in en_stop]
    tokens = [word for word in tokens if len(word) > 2]
    tokens = [get_lemma(token) for token in tokens]
    
    text_data.append(tokens)
    counter += len(tokens)
    index+=1
    
print('Words in text data: ', counter)
numbers_set = set(i for j in text_data for i in j)
print('Unique words:', len(numbers_set))
print()

for tokens in text_data:
    print(tokens)
    print()

Words in text data:  17849
Unique words: 4789

['anyone', 'get', 'blister', 'foot', 'walking', 'much', 'around', 'school']

['month', 'since', 'whole', 'sexual', 'harassment', 'scandal', 'rock', 'ateneo', 'silence', 'deafen', 'prof', 'vocal', 'issue', 'like', 'martial', 'law', 'etc', 'never', 'spoke', 'particular', 'issue', 'opening', 'fucking', 'upsilon', 'scandal', 'like', 'worst', 'shit', 'ever', 'fuck', 'hypocrite']

['think', 'reason', 'really', 'hate', 'filipino', 'college', 'language', 'topic', 'understand', 'filipino', 'learning', 'speak', 'deep', 'tagalog', 'learning', 'like', 'street', 'tagalog', 'culture', 'anything', 'relate', 'present', 'times', 'instead', 'college', 'filipino', 'least', 'experience', 'unnecessary', 'lack', 'better', 'term', 'useless', 'topic', 'lol', 'filipino', 'learn', 'girl', 'fall', 'love', 'nature', 'getting', 'horny', 'learn', 'ancient', 'filipino', 'super', 'weird', 'culture', 'require', 'memorize', 'author', 'book', 'probably', 'never', 'read', 't


['dumb', 'fuck', 'left', 'car', 'double', 'park', 'swimming', 'pool', 'area', 'blocking', 'three', 'car', 'leaving', 'assume', 'driver', 'car', 'sticker', 'know', 'owner', 'car', 'please', 'tell', 'fire', 'driver']

['jsec', 'challenge', 'lit', 'okawari', 'make', 'thank', 'motivational', 'sticker', 'thank', 'tasty', 'meal', 'well', 'hope', 'guy', 'win', 'see', 'guy', 'jsec', 'torikatsu', 'tonkatsu', 'girl', 'roving', 'giving', 'sticker', 'cute', 'whats', 'namee', 'plzzz']

['frightening']

['admu', 'freedom', 'wall']

['alumnus', 'management', 'position', 'yet', 'completely', 'flunk', 'accounting', 'thus', 'failing', 'management', 'people', 'would', 'say', 'lucky', 'position', 'obviously', 'failing', 'prop', 'ateneo', 'powerful', 'subject', 'bull', 'modicum', 'trust', 'sort', 'generate', 'lack', 'connection', 'make', 'management', 'lack', 'technical', 'skill', 'synthesis', 'killing', 'part', 'wish', 'previous', 'metaphor', 'fact', 'reality', 'amount', 'salary', 'job', 'security', 'rep

In [7]:
model = gensim.models.Word2Vec(text_data, size=100, window=10, min_count=1, workers=4)
model.train(text_data, total_examples=len(text_data),epochs=10)

(167547, 178490)

In [8]:
word = 'school'
print('Word:', word)
model.wv.most_similar(positive=word)

Word: school


[('take', 0.9999724626541138),
 ('even', 0.9999715685844421),
 ('get', 0.999970555305481),
 ('day', 0.9999698996543884),
 ('use', 0.9999688863754272),
 ('ateneo', 0.9999686479568481),
 ('also', 0.9999672770500183),
 ('think', 0.999966025352478),
 ('best', 0.9999656677246094),
 ('student', 0.9999656677246094)]

## TF-IDF

In [9]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data)
feature_names = vectorizer.get_feature_names()

In [10]:
tf_idf_list = []
for i in range(len(data)):
    sentence_dict = {}
    feature_index = tfidf_matrix[i,:].nonzero()[1]
    tfidf_scores = zip(feature_index, [tfidf_matrix[i, x] for x in feature_index])
    for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
        sentence_dict[w] = s
    tf_idf_list.append(sentence_dict)

## Sentiment Analysis

In [11]:
for tokens in text_data:
    print(tokens)
    sum_score = 0
    if not tokens:
        continue
    for token in tokens:
        if token in positive_words:
            print("Word: " + token + ", score: 1")
            sum_score += 1
        elif token in negative_words:
            print("Word: " + token + ", score: -1")
            sum_score -= 1
        elif token in neutral_words:
            print("Word: " + token)
            neighbors = model.wv.most_similar(positive=token, topn=100)
            for word in neighbors:
                if word[0] in positive_words:
                    print("Nearest labelled word: " + word[0] + ", score: " + str(word[1]))
                    sum_score += word[1]
                    break
                elif word[0] in negative_words:
                    print("Nearest labelled word: " + word[0] + ", score: -" + str(word[1]))
                    sum_score -= word[1]
                    break
    print("\nAggregate score:", sum_score/len(tokens))
    print()

['anyone', 'get', 'blister', 'foot', 'walking', 'much', 'around', 'school']
Word: anyone
Nearest labelled word: cute, score: 0.9999557733535767
Word: get
Nearest labelled word: good, score: 0.9999760389328003
Word: blister, score: -1
Word: foot
Nearest labelled word: hurt, score: -0.9996961355209351
Word: walking
Nearest labelled word: mean, score: -0.9996867179870605
Word: much
Nearest labelled word: better, score: 0.9999567270278931
Word: around
Nearest labelled word: best, score: 0.9999627470970154
Word: school
Nearest labelled word: best, score: 0.9999656677246094

Aggregate score: 0.2500542625784874

['month', 'since', 'whole', 'sexual', 'harassment', 'scandal', 'rock', 'ateneo', 'silence', 'deafen', 'prof', 'vocal', 'issue', 'like', 'martial', 'law', 'etc', 'never', 'spoke', 'particular', 'issue', 'opening', 'fucking', 'upsilon', 'scandal', 'like', 'worst', 'shit', 'ever', 'fuck', 'hypocrite']
Word: month
Nearest labelled word: fucking, score: -0.9999352097511292
Word: since
Near

Nearest labelled word: cute, score: 0.9996836185455322
Word: connect
Nearest labelled word: issue, score: -0.998248815536499
Word: stable, score: 1
Word: internet
Nearest labelled word: shit, score: -0.9998117089271545
Word: everywhere
Nearest labelled word: cute, score: 0.9998274445533752
Word: lagi
Nearest labelled word: best, score: 0.9996576905250549
Word: nagdadala
Nearest labelled word: beautiful, score: 0.9977445006370544
Word: mobile
Nearest labelled word: bobo, score: -0.9997735023498535
Word: wifi
Nearest labelled word: shit, score: -0.9997524619102478
Word: personal
Nearest labelled word: best, score: 0.9997653961181641
Word: hotspot
Nearest labelled word: cool, score: 0.9990070462226868
Word: accept
Nearest labelled word: good, score: 0.9997365474700928
Word: school
Nearest labelled word: best, score: 0.9999656677246094
Word: wifi
Nearest labelled word: shit, score: -0.9997524619102478
Word: trustable, score: 1
Word: fair, score: 1
Word: take
Nearest labelled word: good, sc

Nearest labelled word: shit, score: -0.9999712109565735
Word: oldie
Nearest labelled word: honest, score: 0.9759252071380615
Word: assume
Nearest labelled word: hard, score: -0.9998152852058411
Word: refer
Nearest labelled word: napapagod, score: -0.9884992241859436
Word: honest, score: 1
Word: remain
Nearest labelled word: good, score: 0.9998509883880615
Word: member
Nearest labelled word: kind, score: 0.999852180480957
Word: still
Nearest labelled word: best, score: 0.9999674558639526
Word: somehow
Nearest labelled word: peace, score: 0.9954122304916382
Word: love, score: 1
Word: work
Nearest labelled word: good, score: 0.9999486804008484
Word: still
Nearest labelled word: best, score: 0.9999674558639526
Word: good, score: 1
Word: friend
Nearest labelled word: fucking, score: -0.9999492168426514
Word: cliquish
Nearest labelled word: nasty, score: -0.9906002283096313
Word: completely
Nearest labelled word: hate, score: -0.9993754029273987
Word: suck, score: -1
Word: sense
Nearest labe

Nearest labelled word: great, score: 0.999956488609314
Word: call
Nearest labelled word: good, score: 0.999957263469696
Word: home
Nearest labelled word: great, score: 0.9999470114707947
Word: years
Nearest labelled word: fuck, score: -0.9999440908432007
Word: grandkids
Nearest labelled word: wrong, score: -0.9975091218948364
Word: privilege
Nearest labelled word: insane, score: -0.9988416433334351
Word: archipelago
Nearest labelled word: ganda, score: 0.9992341995239258
Word: province
Nearest labelled word: hope, score: 0.9996566772460938
Word: another
Nearest labelled word: shit, score: -0.9999523758888245
Word: state
Nearest labelled word: hate, score: -0.9998816847801208
Word: archipelago
Nearest labelled word: ganda, score: 0.9992341995239258
Word: philippines
Nearest labelled word: mean, score: -0.9997822046279907
Word: cease
Nearest labelled word: wrong, score: -0.9988672137260437
Word: exist
Nearest labelled word: great, score: 0.9999418258666992
Word: every
Nearest labelled wo

Nearest labelled word: better, score: 0.9999201893806458
Word: brainwashing, score: -1
Word: people
Nearest labelled word: better, score: 0.9999725222587585
Word: work
Nearest labelled word: good, score: 0.9999486804008484
Word: minimum
Nearest labelled word: gago, score: -0.9996861219406128
Word: wage
Nearest labelled word: gago, score: -0.9997252225875854
Word: job
Nearest labelled word: good, score: 0.9999447464942932
Word: die, score: -1
Word: capitalist
Nearest labelled word: care, score: 0.9990066289901733
Word: system
Nearest labelled word: great, score: 0.999742865562439
Word: instead
Nearest labelled word: great, score: 0.9999477863311768
Word: know
Nearest labelled word: shit, score: -0.9999595880508423
Word: try
Nearest labelled word: best, score: 0.9999566078186035
Word: target
Nearest labelled word: kind, score: 0.9996236562728882
Word: specific
Nearest labelled word: best, score: 0.9999129772186279
Word: audience
Nearest labelled word: hurt, score: -0.999518096446991
Word

...

Nearest labelled word: mean, score: -0.999749481678009
Word: reading
Nearest labelled word: good, score: 0.9999345541000366

Aggregate score: -0.1656266748905182

['girl', 'drive', 'small', 'car', 'big', 'dick', 'energy']
Word: girl
Nearest labelled word: shit, score: -0.9999628663063049
Word: drive
Nearest labelled word: unnecessary, score: -0.984306812286377
Word: small
Nearest labelled word: good, score: 0.9997661113739014
Word: car
Nearest labelled word: fuck, score: -0.9999597072601318
Word: big
Nearest labelled word: cute, score: 0.9999260902404785
Word: dick
Nearest labelled word: hell, score: -0.9987843632698059
Word: energy
Nearest labelled word: shit, score: -0.9998601675033569

Aggregate score: -0.4261688164302281

['atenean', 'sjws', 'easily', 'bait', 'point', 'reaction', 'hilarious']
Word: atenean
Nearest labelled word: shit, score: -0.9998534321784973
Word: sjws
Nearest labelled word: die, score: -0.9990742802619934
Word: easily
Nearest labelled word: advantage, score: 0.

Word: acet
Nearest labelled word: hurt, score: -0.9990142583847046
Word: upcat
Nearest labelled word: mean, score: -0.9993959665298462
Word: even
Nearest labelled word: good, score: 0.999975860118866
Word: among
Nearest labelled word: fail, score: -0.998919665813446
Word: top
Nearest labelled word: shit, score: -0.9999474287033081
Word: upcat
Nearest labelled word: mean, score: -0.9993959665298462
Word: passer
Nearest labelled word: noise, score: -0.9906497001647949
Word: choose
Nearest labelled word: gago, score: -0.9996157884597778
Word: ateneo
Nearest labelled word: fuck, score: -0.9999685287475586
Word: believe
Nearest labelled word: best, score: 0.9999514818191528
Word: best, score: 1
Word: school
Nearest labelled word: best, score: 0.9999656677246094
Word: foreigner
Nearest labelled word: happy, score: 0.9988287091255188
Word: think
Nearest labelled word: best, score: 0.999975860118866
Word: ranking
Nearest labelled word: hard, score: -0.9995536804199219
Word: school
Nearest labe

Word: solid
Nearest labelled word: shit, score: -0.9997636079788208
Word: lion
Nearest labelled word: resent, score: -0.9760982990264893
Word: orgs
Nearest labelled word: best, score: 0.9997714757919312

Aggregate score: -0.3253634770711263

['anong', 'gagawin', 'sinisiraan', 'lalaki', 'gusto', 'sayo']
Word: anong
Nearest labelled word: great, score: 0.9995124936103821
Word: gagawin
Nearest labelled word: professional, score: 0.9917135238647461
Word: sinisiraan, score: -1
Word: lalaki
Nearest labelled word: fool, score: -0.99312824010849
Word: gusto
Nearest labelled word: fuck, score: -0.9999587535858154
Word: sayo
Nearest labelled word: tangina, score: -0.9999244213104248

Aggregate score: -0.333630899588267

['god', 'king', 'destroyer', 'world', 'rammus']
Word: king
Nearest labelled word: die, score: -0.9989659190177917
Word: destroyer
Nearest labelled word: upset, score: -0.9919476509094238
Word: world
Nearest labelled word: good, score: 0.9999338388442993
Word: rammus
Nearest label

Nearest labelled word: cute, score: 0.9997855424880981
Word: building
Nearest labelled word: fuck, score: -0.9999484419822693
Word: isko
Nearest labelled word: better, score: 0.9999074935913086
Word: enemy
Nearest labelled word: good, score: 0.9993722438812256
Word: cannot
Nearest labelled word: best, score: 0.9998729228973389
Word: hide
Nearest labelled word: fuck, score: -0.9998634457588196
Word: matter
Nearest labelled word: mean, score: -0.9999282360076904
Word: form
Nearest labelled word: fucking, score: -0.9999439716339111
Word: choose
Nearest labelled word: gago, score: -0.9996157884597778
Word: wear
Nearest labelled word: great, score: 0.9998518228530884
Word: let
Nearest labelled word: better, score: 0.9999656677246094
Word: true, score: 1
Word: self
Nearest labelled word: better, score: 0.9998259544372559
Word: reveal
Nearest labelled word: sick, score: -0.9971435070037842
Word: elitista
Nearest labelled word: great, score: 0.9995680451393127
Word: elitista
Nearest labelled w

Nearest labelled word: best, score: 0.9999344348907471
Word: mental
Nearest labelled word: better, score: 0.9996816515922546
Word: health
Nearest labelled word: best, score: 0.9997853636741638
Word: though
Nearest labelled word: best, score: 0.9999529719352722

Aggregate score: -0.096112178750785

['swear', 'doods', 'sao', 'gonna', 'best', 'anime', 'end', 'season']
Word: swear
Nearest labelled word: great, score: 0.9996433854103088
Word: doods
Nearest labelled word: thanks, score: 0.9946463108062744
Word: best, score: 1
Word: anime
Nearest labelled word: cute, score: 0.9998393058776855
Word: end
Nearest labelled word: hope, score: 0.9999582171440125
Word: season
Nearest labelled word: fuck, score: -0.9995455145835876

Aggregate score: 0.4993177130818367

['tag', 'crush', 'crush']
Word: tag
Nearest labelled word: cute, score: 0.9998632669448853
Word: crush
Nearest labelled word: tangina, score: -0.9999052882194519
Word: crush
Nearest labelled word: tangina, score: -0.9999052882194519

A